## Auditing and Exporting Detections

In [1]:
import os
import argparse
import numpy as np
import json
import csv
import rasterio

# shapely.geometry import mapping, Polygon
# import fiona # only required for exporting to shapefiles

In [23]:
# ingest the image
#infile = "mosaics/SteepleJason_Bubble_Nov2019_transparent_mosaic_group1.tif"

#img_dir = infile.split(".")[0]
#prj_name = img_dir.split("/")[-1]

infile = "mosaics/SteepleJason_Hump_Nov2019_transparent_mosaic_group1.tif"

img_dir = infile.split(".")[0]
#prj_name = img_dir.split("/")[-1]

#infile = "mosaics/SteepleJason_Blob_Nov2019_transparent_mosaic_group1.tif"

#img_dir = infile.split(".")[0]
#prj_name = img_dir.split("/")[-1]

In [24]:
img_dir

'mosaics/SteepleJason_Hump_Nov2019_transparent_mosaic_group1'

In [25]:
# open the output from detection

with open('detections/detections_hump.json') as f:
    detected_labels = json.load(f)

In [26]:
# open the previous training data

#with open(img_dir + '/via_project_18Mar2020_11h08m45s.json') as f:
    #existing_labels = json.load(f)

In [27]:
len(detected_labels)

352

### Exporting to Shapefile

In [28]:
image_annotations = []
for key, value in detected_labels.items():
    #print(key)
    annotation = [[key][0].split("/")[-1]]
    detections = []
    for item in value:
        box = item['box']
        detections.append(box)
        print(item)
    annotation.append(detections)
    image_annotations.append(annotation)

{'box': [428, 290, 502, 388], 'label': 0, 'score': 0.9897358417510986}
{'box': [740, 222, 833, 319], 'label': 0, 'score': 0.9854877591133118}
{'box': [216, 243, 293, 344], 'label': 0, 'score': 0.9852532744407654}
{'box': [519, 500, 585, 608], 'label': 0, 'score': 0.985191285610199}
{'box': [68, 42, 156, 142], 'label': 0, 'score': 0.9841992855072021}
{'box': [376, 691, 465, 786], 'label': 0, 'score': 0.9810066223144531}
{'box': [841, 658, 947, 733], 'label': 0, 'score': 0.9742249846458435}
{'box': [789, 810, 892, 876], 'label': 0, 'score': 0.972902774810791}
{'box': [628, 378, 693, 482], 'label': 0, 'score': 0.9722842574119568}
{'box': [100, 385, 173, 494], 'label': 0, 'score': 0.9713854789733887}
{'box': [153, 598, 256, 680], 'label': 0, 'score': 0.9690900444984436}
{'box': [171, 812, 236, 916], 'label': 0, 'score': 0.9682465195655823}
{'box': [359, 133, 426, 224], 'label': 0, 'score': 0.9662970304489136}
{'box': [323, 899, 437, 964], 'label': 0, 'score': 0.9595823884010315}
{'box': [3

{'box': [744, 58, 862, 149], 'label': 0, 'score': 0.9163042902946472}
{'box': [588, 196, 702, 263], 'label': 0, 'score': 0.8642733693122864}
{'box': [177, 836, 279, 916], 'label': 0, 'score': 0.7674985527992249}
{'box': [839, 427, 927, 530], 'label': 0, 'score': 0.7164090275764465}
{'box': [196, 56, 283, 147], 'label': 0, 'score': 0.7030090689659119}
{'box': [488, 20, 602, 97], 'label': 0, 'score': 0.6579554080963135}
{'box': [652, 490, 777, 557], 'label': 0, 'score': 0.6416859030723572}
{'box': [440, 182, 514, 283], 'label': 0, 'score': 0.5885396599769592}
{'box': [578, 365, 653, 460], 'label': 0, 'score': 0.5284615159034729}
{'box': [6, 871, 101, 974], 'label': 0, 'score': 0.509193480014801}
{'box': [241, 293, 352, 375], 'label': 0, 'score': 0.9812048673629761}
{'box': [51, 106, 168, 171], 'label': 0, 'score': 0.9713870286941528}
{'box': [843, 383, 954, 450], 'label': 0, 'score': 0.9951260685920715}
{'box': [374, 639, 446, 750], 'label': 0, 'score': 0.9868757724761963}
{'box': [453, 

{'box': [443, 752, 514, 845], 'label': 0, 'score': 0.7480415105819702}
{'box': [835, 897, 918, 983], 'label': 0, 'score': 0.6611320376396179}
{'box': [67, 851, 165, 931], 'label': 0, 'score': 0.9884510040283203}
{'box': [82, 221, 186, 292], 'label': 0, 'score': 0.9881638884544373}
{'box': [579, 420, 653, 524], 'label': 0, 'score': 0.9862725734710693}
{'box': [471, 190, 576, 254], 'label': 0, 'score': 0.9825373291969299}
{'box': [244, 398, 339, 477], 'label': 0, 'score': 0.978209376335144}
{'box': [647, 623, 739, 711], 'label': 0, 'score': 0.9746599793434143}
{'box': [864, 306, 931, 412], 'label': 0, 'score': 0.9741877317428589}
{'box': [629, 849, 713, 946], 'label': 0, 'score': 0.9707372188568115}
{'box': [714, 204, 801, 300], 'label': 0, 'score': 0.9680764675140381}
{'box': [845, 652, 929, 732], 'label': 0, 'score': 0.9649242758750916}
{'box': [263, 764, 343, 864], 'label': 0, 'score': 0.9627877473831177}
{'box': [417, 624, 479, 725], 'label': 0, 'score': 0.9622156023979187}
{'box': [

In [29]:
# ingest back in the coordinates of detections within an image referenced by their filename

with open('mosaics/data_hump.json') as f:
    img_data = json.load(f)

# open the satellite image
dataset = rasterio.open(infile)
#dataset = rasterio.open(infile_hump)
#dataset = rasterio.open(infile_blob)
    
geolocated_annotations = []
for annotation in image_annotations:
    for detection in annotation[1]:
        try:            
            local_bounding_box = np.array([[detection[0], detection[1]], [detection[2], detection[1]], [detection[2], detection[3]], [detection[0], detection[3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]

            geolocated_bb = []
            for point in image_located_bb:
                geolocated_bb.append(dataset.transform * point)
            geolocated_annotations.append(geolocated_bb)
        except ValueError: # if the image doesn't have a detection
            pass

In [30]:
len(geolocated_annotations)

5701

In [31]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str')
  ])
}

with fiona.open(
    'detections_hump.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(geolocated_annotations):
        record = {
            'geometry': {'coordinates': [polygon], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', infile),
                                       ('Detection', 'Albatross')
                                       ]),
            'type': 'Feature'}
        c.write(record)
        

### RetinaNet to Existing VIA

In [6]:
# add the new detections to the old via_region_data.json file

#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    for filename_size, metadata in existing_labels.items():
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                #print(x1,x2,y1,y2)
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})
    

In [7]:
# write out new VIA file with additional detections

with open(img_dir + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)